In [226]:
import pandas as pd
import numpy as np
from modules import *
from nltk.corpus import stopwords
from nltk.stem.porter import *
from nltk import word_tokenize
from konlpy.tag import Okt
from nltk.util import everygrams

In [227]:
#원하는 태그 추출 & 불용어 제거
okt = Okt()

#불용어
stopwords = pd.read_csv('ko-stopwords.csv') #한국어불용어사전 불러오기
stopwords=list(stopwords['stopwords']) 
stopwords.extend(['에서','고','이다','는','한','씨', "것","거","게","데","이다","건","고","되다","되어다","걸","기",
                  "시","네","듯","랍니","중이","얘","스","도도", "나","수","개","내","기","제","저","인","있다","이렇다",
                  "그렇다","번","위","팅","분","인","링","란","포","두", "진짜", "하다" ,"이다" ,"가다", "이제" ,"들다",
                 '에서','고','이다','젛','뇨','껀데','뭘'
                 ,'최고','느리다','마을','최고','항상','포장','재다','말다','목','넘김','만족하다','정말','역시','아주','감사하다','오다','자주','요','늘','넘다','않다','더','병','다','숙성','나다','술','드리다','마트','꼼꼼하다','잇다'
                 ,'복','순도','용','비싸다','가격','많다','강하다','보내다','추천','보고','쏘다','느낌','사다','처음','많이','톡','생각','엄청','날','아니다'
                 ,'믿다', '구입', '사먹다', '한번', '싶다', '콤', '완전', '제품', '재', '해주다', '말', '사람','특별하다','언제나','빨르다','두번째','전','계속','달'
                 ,'추석', '인생', '최애', '명절', '해', '중', '생각나다', '시원하다', '다르다', '알다', '가족', '정도', '기대', '친구', '괜찮다', '부모님', '지인', '아직'
                 ,'만족스럽다', '빼다', '상품', '굿', '못', '도', '기분', '다시', '주다', '즐기다', '맞다', '맛보다', '제일', '집', '입맛', '모르다', '먹기'
                 ,'조심하다', '따다', '차다', '엄마', '예정', '만족', '감', '일반', '파다', '막', '근데', '별로', '적당하다','예전', '가끔', '유명하다', '만', '뚜껑', '매우', '기대하다', '의사', '한잔', '신선하다', '느끼다', '그냥', '아쉽다'
                 ,'오픈', '쎄다', '여자', '호불호', '꼭', '벌써', '버리다', '이렇게', '돼다', '크다', '느껴지다','고급', '종종', '어울리다', '알', '남편', '빨리', '어른', '소문', '아버지', '마시기', '할인', '살다' 
                 ,'풍부하다', '모임', '해드리다', '빠지다', '나서다', '원래', '아빠', '걸리다', '안', '도착', '없이'
                 ,'하루', '딱이다', '상태', '향', '후', '설', '주변', '사서', '때문', '반하다', '연말'
                 ,'해봤다', '강추', '찾다', '자꾸', '살짝', '몇번', '빨', '요즘', '신세계', '사진', '생일'
                 ,'빠르다', '자다', '구매', '마시다', '받다', '보다', '없다', '새다', '숙취', '아스파탐', '대포', '넘어가다','주가'
                 ,'지나다', '무', '머리', '편하다', '아프다', '두다', '인터넷', '택배', '숙성하다', '일주일', '봄', '배달', '만들다', '애용', '냉장고' 
                 ,'여름', '잔', '앞', '며칠', '팔다', '달라지다', '가장', '변하다', '번창', '안전하다', '젤', '배상면', '건강하다', '굿굿'
                 ,'따르다', '이용', '확실하다', '날짜', '시중', '들어가다', '첨가', '유통', '아이스팩', '제조', '넣다', '생', '다양하다','종류','들어가다','전통주','마켓','술술','후기','사보다','부담','여러가지','리뷰','블루'
                 ,'인기','궁금하다','대상','무난','취향','적다','단','평이','이용','터지다','맘'
                 ]
                 +'동학 저렴하다 청 서비스 술잔 소주 강쇠주 오메 기술 조선 대비 요리 사용 데우다 조사 쓰다 감사 음식 알콜 경험 청주 강쇠 가성 전주 모주 뜨다 신기하다 화주 좋아하다 주문 예쁘다 이쁘다 얼음 산 토닉 워터 빨대 깨지다 빙 탄복 박스 약하다 국산 오프너 위트 소백산 제공 분위기 아내 크리스마스 마음 마루 베베 솔직하다 초보자 와이프 작성 위트 디자인 굳다 달다 시큼하다 달달 선물 깔끔하다 뒤 비비 이 있 하 것 들 그 되 수 이 보 않 없 나 사람 주 아니 등 같 우리 때 년 가 한 지 대하 오 말 일 그렇 위하 기한 달라 첫날 첨 주말 걱정 전이 이유 해보다 저번 보지 함 기대다 상 비 베스트 도수 음 존맛 개인'.split()
                 ) #불용어 추가
stopwords=set(stopwords) # 중복제거    

# 형태소 분석 함수 만들기
def okt_pos_tagging2(string):
    pos_words = okt.pos(string, stem=True, norm=True) # 형태소 분석. 단어는 사전형으로 바꿔주기
    words = [word for word, tag in pos_words if tag 
             in ['Noun', 'Adjective', 'Verb','Adverb'] if word not in stopwords ]

    # n_gram 만들기
    egram = list(everygrams(words, min_len=1, max_len=1))
    egram_token = [' '.join(grams) for grams in egram]

    return egram_token

def okt_pos_tagging(string):
    pos_words = okt.pos(string, stem=True, norm=True) # 형태소 분석. 단어는 사전형으로 바꿔주기
    words = [word for word, tag in pos_words if tag 
             in ['Noun', 'Adjective', 'Verb','Adverb'] if word not in stopwords ]

    return words


In [228]:
# 불러올 csv 이름 리스트
# 전체 리뷰를 불러온후 tf-idf 분석
# name_list =["술 리뷰(복순도가)","느린마을 막걸리","술 리뷰(죽향도가)"]
# name_list =["금이산농원_복숭아와인","덕유양조_산머루주","배상면주가_빙탄복","산내들_소백산스위트와인","컨츄리와인_영동포도와인"]
name_list =["청주_고헌정_동학1957_특선","청주_양촌양조_무농약_우렁이쌀","청주_오메기맑은술","청주_우포의아침","탁주_모주","탁주_이화주"]

In [229]:
all_data = pd.DataFrame()

for name in name_list:
    try: 
        data = pd.read_csv(f"{name}.csv", encoding='utf-8-sig')
    except:
        data = pd.read_csv(f"{name}.csv", encoding='cp949')
    # 전처리
    data.drop_duplicates(['text'],ignore_index=True, inplace=True) # 중복 제거
    try:
        data.columns = ['date', 'star','doc']
    except:
        data.columns = ['date', 'star','doc','name']
    data.doc = data.doc.str.replace("[^가-힣 ]"," ") # 한글만 남기고 특수문자, 숫자, 영어 삭제
    data.doc = data.doc.str.replace("\s+", " ") # white space 삭제
    data.doc = data.doc.str.replace("럽고목","럽고 목")
    data.doc = data.doc.str.replace("부 드럽","부드럽")
    data.doc = data.doc.str.replace("가성", "가성비")
    data.dropna(inplace=True)
    data.reset_index(drop=True, inplace=True)
    # globals()["감홍로 40도"] 등으로 데이터 프레임 저장됨
    globals()[f"{name}"] = data
    all_data = pd.concat([all_data,data],axis=0)


C:\Users\user\AppData\Local\Temp\ipykernel_16648\4028454450.py:14: FutureWarning: The default value of regex will change from True to False in a future version.
  data.doc = data.doc.str.replace("[^가-힣 ]"," ") # 한글만 남기고 특수문자, 숫자, 영어 삭제
C:\Users\user\AppData\Local\Temp\ipykernel_16648\4028454450.py:15: FutureWarning: The default value of regex will change from True to False in a future version.
  data.doc = data.doc.str.replace("\s+", " ") # white space 삭제
C:\Users\user\AppData\Local\Temp\ipykernel_16648\4028454450.py:14: FutureWarning: The default value of regex will change from True to False in a future version.
  data.doc = data.doc.str.replace("[^가-힣 ]"," ") # 한글만 남기고 특수문자, 숫자, 영어 삭제
C:\Users\user\AppData\Local\Temp\ipykernel_16648\4028454450.py:15: FutureWarning: The default value of regex will change from True to False in a future version.
  data.doc = data.doc.str.replace("\s+", " ") # white space 삭제
C:\Users\user\AppData\Local\Temp\ipykernel_16648\4028454450.py:14: FutureWarning

In [230]:
all_data

,date,star,doc,name
0,2022-11-12,5,몇 달 전에 맛보고 생각나 다시 구매한 술입니다그동안 술을 바꿔가며 여러 청주를 마...,NaN
1,2022-11-16,5,먼저 미안하게도 구매확정에 대해 인지하지 못해서 자동구매확정이 되었네요 개를 주문했...,NaN
2,2022-11-26,5,전통주도 종류별로 많고 맛도 다양해서 좋네요빠른배송과 꼼꼼한 포장으로 잘 받았습니다,NaN
3,2022-11-23,5,가족들하고 맛있게 잘 먹었어요술 잘 마시는 분들은 끝도없이 들어가겠어요,NaN
4,2022-11-26,5,막걸리 유통기한도 길고 포장도 엄청 꼼꼼히 해주셨어요 맛있어요,NaN
...,...,...,...,...
1611,2017-10-01,5,그냥 먹어도 그냥 물에 타 먹어도 그냥단 새롭다능,술샘 이화주 8도
1612,2017-09-18,5,아버지께서 벌써 반통 비우셨네요,술샘 이화주 8도
1613,2017-09-10,5,아버지선물로 드렸는데 좋아하시네요 좋습니다,술샘 이화주 8도
1614,2017-09-11,5,간강한 맛 좋네요,술샘 이화주 8도


In [231]:
from sklearn.feature_extraction.text import TfidfVectorizer

tfidf = TfidfVectorizer(tokenizer=okt_pos_tagging)
tfidf_fit = tfidf.fit(all_data.doc)


c:\Users\user\AppData\Local\Programs\Python\Python38\lib\site-packages\sklearn\feature_extraction\text.py:524: UserWarning: The parameter 'token_pattern' will not be used since 'tokenizer' is not None'
  warnings.warn(


In [232]:
# 특징을 추출할 문서의 tf-idf를 구함
# "술 리뷰(복순도가)" : 110,"느린마을 막걸리" : 130,"술 리뷰(죽향도가)" : 300
a = "청주_고헌정_동학1957_특선"
bow_rep_tfidf = tfidf_fit.transform(globals()[a].doc)
word_count = pd.DataFrame({
    '단어': tfidf.get_feature_names(),
    'tf-idf': bow_rep_tfidf.sum(axis=0).flat,
    'idf': tfidf.idf_
})

c:\Users\user\AppData\Local\Programs\Python\Python38\lib\site-packages\sklearn\utils\deprecation.py:87: FutureWarning: Function get_feature_names is deprecated; get_feature_names is deprecated in 1.0 and will be removed in 1.2. Please use get_feature_names_out instead.
  warnings.warn(msg, category=FutureWarning)


In [233]:
# idf 가 높을수록 잘 안나오는 단어
# tf-idf 가 높을수록 중요한 단어
# 두값을 적절히 조정해서 특징 추출

word_count.sort_values('tf-idf',ascending=False).head(50)[word_count['idf']>3.5][word_count['tf-idf'] * (len(all_data)/len(globals()[a]))>50]

C:\Users\user\AppData\Local\Temp\ipykernel_16648\751632845.py:5: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  word_count.sort_values('tf-idf',ascending=False).head(50)[word_count['idf']>3.5][word_count['tf-idf'] * (len(all_data)/len(globals()[a]))>50]


,단어,tf-idf,idf
1539,부드럽다,66.150791,3.526587
1688,사케,15.161386,5.576428
20,가볍다,13.543217,5.576428


In [234]:
a = ["청주_고헌정_동학1957_특선","청주_양촌양조_무농약_우렁이쌀","청주_오메기맑은술","청주_우포의아침","탁주_모주","탁주_이화주"]
Tags = {"donghak": [],"yangchon": [],"omegi": [],"upo": [],"moju": [],"ihwa": []}
keys = ["donghak","yangchon","omegi","upo","moju","ihwa"]
for i in range(len(a)):
    bow_rep_tfidf = tfidf_fit.transform(globals()[a[i]].doc)
    word_count = pd.DataFrame({
        '단어': tfidf.get_feature_names(),
        'tf-idf': bow_rep_tfidf.sum(axis=0).flat,
        'idf': tfidf.idf_
    })
    
    df = word_count.sort_values('tf-idf',ascending=False).head(50)[word_count['idf']>3.5][word_count['tf-idf'] * (len(all_data)/len(globals()[a[i]]))>50]

    word_count.sort_values('tf-idf',ascending=False).head(50)[word_count['idf']>3.5][word_count['tf-idf'] * (len(all_data)/len(globals()[a[i]]))>50].to_csv(f'청주_모주_특징{i}.csv')

    # for i in range(len(Tags)):
    for j in df['단어']:
        Tags[keys[i]].append(j)
Tags['donghak']

c:\Users\user\AppData\Local\Programs\Python\Python38\lib\site-packages\sklearn\utils\deprecation.py:87: FutureWarning: Function get_feature_names is deprecated; get_feature_names is deprecated in 1.0 and will be removed in 1.2. Please use get_feature_names_out instead.
  warnings.warn(msg, category=FutureWarning)
C:\Users\user\AppData\Local\Temp\ipykernel_16648\2793170590.py:12: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  df = word_count.sort_values('tf-idf',ascending=False).head(50)[word_count['idf']>3.5][word_count['tf-idf'] * (len(all_data)/len(globals()[a[i]]))>50]
C:\Users\user\AppData\Local\Temp\ipykernel_16648\2793170590.py:14: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  word_count.sort_values('tf-idf',ascending=False).head(50)[word_count['idf']>3.5][word_count['tf-idf'] * (len(all_data)/len(globals()[a[i]]))>50].to_csv(f'청주_모주_특징{i}.csv')


['부드럽다', '사케', '가볍다']

In [235]:
# # 특징을 추출할 문서의 tf-idf를 구함
# # a = ["술 리뷰(복순도가)","느린마을 막걸리","술 리뷰(죽향도가)"]
# # Tags = {"boksundoga": [],"maeul": [],"jukhyang": []}
# # keys = ["boksundoga","maeul","jukhyang"]

# a = ["금이산농원_복숭아와인","덕유양조_산머루주","배상면주가_빙탄복","산내들_소백산스위트와인","컨츄리와인_영동포도와인"]
# Tags = {"nongwon": [],"duckyang": [],"baesang": [],"sobak": [],"yungdong": []}
# keys = ["nongwon","duckyang","baesang","sobak","yungdong"]
# for i in range(len(a)):
#     bow_rep_tfidf = tfidf_fit.transform(globals()[a[i]].doc)
#     word_count = pd.DataFrame({
#         '단어': tfidf.get_feature_names(),
#         'tf-idf': bow_rep_tfidf.sum(axis=0).flat,
#         'idf': tfidf.idf_
#     })
    
#     df = word_count.sort_values('tf-idf',ascending=False).head(50)[word_count['idf']>3.5][word_count['tf-idf'] * (len(all_data)/len(globals()[a[i]]))>110]

#     word_count.sort_values('tf-idf',ascending=False).head(50)[word_count['idf']>3.5][word_count['tf-idf'] * (len(all_data)/len(globals()[a[i]]))>110].to_csv(f'과실주_특징{i}.csv')

#     # for i in range(len(Tags)):
#     for j in df['단어']:
#         Tags[keys[i]].append(j)
# Tags['nongwon']

In [236]:
Tags['yangchon']

['부드럽다', '약주', '백세주']

In [237]:
df0 = pd.read_csv('청주_모주_특징0.csv')
del df0['Unnamed: 0']
df1 = pd.read_csv('청주_모주_특징1.csv')
del df1['Unnamed: 0']
df2 = pd.read_csv('청주_모주_특징2.csv')
del df2['Unnamed: 0']
df3 = pd.read_csv('청주_모주_특징3.csv')
del df3['Unnamed: 0']
df4 = pd.read_csv('청주_모주_특징4.csv')
del df4['Unnamed: 0']
df5 = pd.read_csv('청주_모주_특징5.csv')
del df5['Unnamed: 0']

df0

,단어,tf-idf,idf
0,부드럽다,66.150791,3.526587
1,사케,15.161386,5.576428
2,가볍다,13.543217,5.576428


In [238]:
df1

,단어,tf-idf,idf
0,부드럽다,29.231682,3.526587
1,약주,8.582859,5.933103
2,백세주,5.590365,7.031715


In [239]:
df2

,단어,tf-idf,idf
0,부드럽다,55.320974,3.526587
1,제주도,22.094058,5.700480
2,와인,20.280304,5.758749
3,화이트와인,15.087080,5.981893
4,단맛,12.816981,5.055652


In [240]:
df3

,단어,tf-idf,idf
0,부드럽다,21.910287,3.526587
1,사케,9.315562,5.576428
2,가볍다,4.645555,5.576428
3,따뜻하다,4.117553,6.780401


In [241]:
df4

,단어,tf-idf,idf
0,음료,9.514760,6.626250
1,막걸리,5.171283,3.859631
2,음료수,4.782423,7.368187
3,진하다,3.594269,5.527638
4,낮다,3.158810,6.675040


In [242]:
df5

,단어,tf-idf,idf
0,막걸리,91.091134,3.859631
1,요거트,51.397384,4.694039
2,특이하다,31.341199,5.511889
3,부드럽다,26.270160,3.526587
4,색다르다,21.216747,5.981893
5,과일,20.386697,5.466080
6,딸기,16.034749,6.205036
7,새롭다,16.031006,5.933103
8,독특하다,14.714407,6.059854


In [243]:
def getHashTagString(list_of_tags):
	hashTagString = ""

	for tag in list_of_tags:
		hashTagString += " #" + tag

	return hashTagString

In [244]:
Tags['donghak']

['부드럽다', '사케', '가볍다']

In [245]:
getHashTagString(Tags['donghak'][0:3])[:].strip()

'#부드럽다 #사케 #가볍다'

In [246]:
getHashTagString(Tags['yangchon'][0:3])[:].strip()

'#부드럽다 #약주 #백세주'

In [247]:
getHashTagString(Tags['omegi'][0:3])[:].strip()

'#부드럽다 #제주도 #와인'

In [248]:
getHashTagString(Tags['upo'][0:3])[:].strip()

'#부드럽다 #사케 #가볍다'

In [249]:
getHashTagString(Tags['moju'][0:3])[:].strip()

'#음료 #막걸리 #음료수'

In [250]:
getHashTagString(Tags['ihwa'][0:3])[:].strip()

'#막걸리 #요거트 #특이하다'